In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import Image, display
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
import pylab as pl
import re
import codecs
import nltk
import pymorphy2
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

%pylab inline
pylab.rcParams['figure.figsize'] = (15,10)

Populating the interactive namespace from numpy and matplotlib


In [61]:
# для оброботки текста

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

-----

# Задание 1

1. С помощью *pd.read_csv()* загрузите датафреймы positive.csv и negative.csv (обратите внимание, что исходные таблицы не содержат наименования столбцов и на первой строке располагаются данные. Файлы расположены в папке datasets);
2. Объедините датафреймы с помощью *pd.concat()* в один датафрейм;
3. Убедитесь, что в новом датафрейме индексация сквозная и без повторов;
4. Переименуйте столбцы датафрейма (столбцы полностью соответствуют примеру);
5. Выведите информацию об общем количестве полученных твитов, сколько из них негативных, сколько позитивных.

-----

In [15]:
col=['id', 'date', 'name', 'text', 'positive', 'rep', 'rtv', 'fav',
       'total_count', 'fol', 'friends', 'list_count']

In [18]:
df_p = pd.read_csv('../datasets/positive.csv', sep=';', header=None)
df_p

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
114906,411368729235054592,1386912922,diminlisenok,"Спала в родительском доме, на своей кровати......",1,0,0,0,1497,56,34,2
114907,411368729424187392,1386912922,qilepocagotu,RT @jebesilofyt: Эх... Мы немного решили сокра...,1,0,1,0,692,225,210,0
114908,411368796537257984,1386912938,DennyChooo,"Что происходит со мной, когда в эфире #proacti...",1,0,0,0,4905,448,193,13
114909,411368797447417856,1386912938,bedowabymir,"""Любимая,я подарю тебе эту звезду..."" Имя како...",1,0,0,0,989,254,251,0


In [13]:
df_n = pd.read_csv('../datasets/negative.csv', sep=';', header=None)
df_n

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0
...,...,...,...,...,...,...,...,...,...,...,...,...
111918,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
111919,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
111920,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
111921,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


In [20]:
df = pd.concat([df_p, df_n], ignore_index=True)
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


In [21]:
df.columns = col

In [22]:
df

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


In [41]:
df.positive[df.positive==-1]=0

In [44]:
print(f"positive True = {df[df['positive'] == 1].shape[0]}")
print(f"positive False = {df[df['positive'] == 0].shape[0]}")

positive True = 114911
positive False = 111923


----

# Задание 2

Произведите очистку данных, сформированных в задании 1. По результатам очистки выведите на экран следующую информацию:   
- Общее количество слов перед удалением слов, встречающихся 1 раз;
- Количество слов, встречающихся 1 раз;
- Итоговое количество слов;
- Количество пустых твитов;
- Из них позитивных твитов;
- Количество твитов после удаления пустых.

----

***Совет:*** сохраняйте промежуточные результаты очистки, чтобы в случае неверных действий на каком-либо этапе не пересчитывать все предыдущие этапы

---

In [46]:
df.text = df.text.str.lower()
df.text = df.text.str.replace(r"[^А-Яа-я]"," ")
df.text

0                      хоть я и школота  но поверь  у на...
1         да  все таки он немного похож на него  но мой ...
2                        ну ты идиотка  я испугалась за ...
3                          кто то в углу сидит и погибае...
4                         вот что значит страшилка    но...
                                ...                        
226829    но не каждый хочет что то исправлять          ...
226830    скучаю так     только             вправляет мо...
226831            вот и в школу  в говно это идти уже надо 
226832                             тауриэль  не грусти  ...
226833    такси везет меня на работу  раздумываю приплат...
Name: text, Length: 226834, dtype: object

In [48]:
df.text = list(map(word_tokenize, df.text))
df.text

0         [хоть, я, и, школота, но, поверь, у, нас, то, ...
1         [да, все, таки, он, немного, похож, на, него, ...
2                [ну, ты, идиотка, я, испугалась, за, тебя]
3         [кто, то, в, углу, сидит, и, погибает, от, гол...
4         [вот, что, значит, страшилка, но, блин, посмот...
                                ...                        
226829         [но, не, каждый, хочет, что, то, исправлять]
226830    [скучаю, так, только, вправляет, мозги, но, я,...
226831    [вот, и, в, школу, в, говно, это, идти, уже, н...
226832                        [тауриэль, не, грусти, обнял]
226833    [такси, везет, меня, на, работу, раздумываю, п...
Name: text, Length: 226834, dtype: object

In [58]:
%%time
russian_stopwords = stopwords.words("russian")
russian_stopwords.sort()

Wall time: 2.79 ms


In [54]:
def delete_stopword(words):
    global russian_stopwords
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

In [57]:
%%time
df.text = list(map(delete_stopword, df.text))

Wall time: 2.89 s


In [59]:
%%time

morph = pymorphy2.MorphAnalyzer()

def lemmatization(words):
    global morph
    new_s = [morph.parse(word)[0].normal_form for word in words]
    return new_s

df.text = list(map(lemmatization, df.text))

Wall time: 4min 20s


In [60]:
ok_lemmatization = df.text

In [62]:
%%time

def to_str(s):
    new_s = ' '.join(j for j in s)
    return new_s

text_tokens = word_tokenize(' '.join(j for j in list(map(to_str, df.text))))
text_tokens

Wall time: 5.82 s


['школотый',
 'поверь',
 'самый',
 'общество',
 'профилировать',
 'предмет',
 'тип',
 'таки',
 'немного',
 'похожий',
 'мальчик',
 'равно',
 'идиотка',
 'испугаться',
 'угол',
 'сидеть',
 'погибать',
 'голод',
 'ещ',
 'порция',
 'взять',
 'хотя',
 'жрать',
 'хотеть',
 'значит',
 'страшилка',
 'блин',
 'посмотреть',
 'часть',
 'создаться',
 'ощущение',
 'автор',
 'курить',
 'любить',
 'знать',
 'бля',
 'ох',
 'это',
 'друг',
 'такой',
 'мимими',
 'ухаж',
 'р',
 'мой',
 'ухо',
 'жр',
 'так',
 'поприветствовать',
 'мой',
 'новый',
 'читатель',
 'частичка',
 'сидней',
 'любить',
 'мама',
 'папа',
 'остальной',
 'влюбляться',
 'привязываться',
 'забыть',
 'вчера',
 'приехать',
 'деревня',
 'наш',
 'котэнить',
 'испугаться',
 'утром',
 'прийти',
 'к',
 'общаться',
 'добрый',
 'задумать',
 'дааааа',
 'вс',
 'равно',
 'обзор',
 'урезать',
 'просто',
 'один',
 'версия',
 'сильно',
 'порезать',
 'другой',
 'полный',
 'мусор',
 'смотреть',
 'случайно',
 'аспирантура',
 'попасть',
 'наука',
 'никт

In [67]:
text = nltk.Text(text_tokens)
fdist = FreqDist(text)
fdist

FreqDist({'это': 20516, 'хотеть': 11828, 'день': 10924, 'такой': 8931, 'сегодня': 8890, 'быть': 8735, 'очень': 7669, 'ты': 7087, 'знать': 6981, 'год': 6863, ...})

In [68]:
%%time

words_to_del = list(filter(lambda k: fdist[k] == 1, fdist))

Wall time: 127 ms


In [74]:
# мега строчка

words_to_del = set(words_to_del)

In [73]:
%%time

def delete_word(words):
    global words_to_del
    new_s = [word for word in words if word not in words_to_del]
    return new_s

df.text = list(map(delete_word, df.text))
df.text = list(map(to_str, df.text))
df.text

Wall time: 1.05 s


0                школотый поверь самый общество предмет тип
1                        таки немного похожий мальчик равно
2                                        идиотка испугаться
3         угол сидеть погибать голод ещ порция взять хот...
4         значит страшилка блин посмотреть часть создать...
                                ...                        
226829                             каждый хотеть исправлять
226830                 скучать вправлять мозг равно скучать
226831                                 школа говно это идти
226832                               тауриэль грусть обнять
226833    такси везти работа раздумывать втащить пятый э...
Name: text, Length: 226834, dtype: object

In [76]:
print(f'Общее количество слов перед удалением слов - {len(text_tokens)}')
print(f'Количество слов, встречающихся 1 раз - {len(words_to_del)}')
print(f'Итоговое количество слов - {len(text_tokens) - len(words_to_del)}')

Общее количество слов перед удалением слов - 1493275
Количество слов, встречающихся 1 раз - 53568
Итоговое количество слов - 1439707


In [84]:
print(f"Количество пустых твитов - {len(df[df.text == ''])}")
print(f"Из них позитивных твитов - {len(df[(df.text == '') & (df.positive == 1)])}")
df = df.drop(df[df.text == ''].index, axis = 0)
print(f'Количество твитов после удаления пустых - {len(df)}')

Количество пустых твитов - 856
Из них позитивных твитов - 382
Количество твитов после удаления пустых - 225978


---

# Задание 3

1. Кодировать данные методом мешка слов.
2. Кодировать данные методом TF-IDF.
3. Построить классификатор на основе логистической регрессии, используя мешок слов.
4. Построить классификатор на основе логистической регрессии, используя TF-IDF.
5. Построить классификатор на основе случайного леса, используя мешок слов.
6. Построить классификатор на основе случайного леса, используя TF-IDF.
7. Сделайте выводы о разработанных классификаторах.

---

При разбиении на обучающую и тестовую выборки, следует указать *test_size=0.3*

---

***Рекомендация:*** для случайного леса параметр n_estimator должен быть не менее 200

In [86]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text, df.positive, test_size=0.2, random_state=21)
len(y_train), y_train.sum(), len(y_test), y_test.sum()

(180782, 91568, 45196, 22961)

In [87]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer()
cv_train = cv.fit_transform(X_train)
cv_test = cv.transform(X_test)

tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(X_train)
tfidf_test = tfidf.transform(X_test)

In [90]:
%%time

lr = LogisticRegression(random_state=21)
lr.fit(cv_train, y_train)
cv_pred = lr.predict(cv_test)
print('\t\t--------------test--------------')
print(classification_report(y_test, cv_pred))
print('\t\t--------------train--------------')
print(classification_report(y_train, lr.predict(cv_train)))

		--------------test--------------
              precision    recall  f1-score   support

           0       0.72      0.71      0.71     22235
           1       0.72      0.73      0.72     22961

    accuracy                           0.72     45196
   macro avg       0.72      0.72      0.72     45196
weighted avg       0.72      0.72      0.72     45196

		--------------train--------------
              precision    recall  f1-score   support

           0       0.80      0.79      0.79     89214
           1       0.79      0.80      0.80     91568

    accuracy                           0.79    180782
   macro avg       0.79      0.79      0.79    180782
weighted avg       0.79      0.79      0.79    180782

Wall time: 2.49 s


In [91]:
lr = LogisticRegression(random_state=21)
lr.fit(tfidf_train, y_train)
tfidf_pred = lr.predict(tfidf_test)
print('\t\t--------------test--------------')
print(classification_report(y_test, tfidf_pred))
print('\t\t--------------train--------------')
print(classification_report(y_train, lr.predict(tfidf_train)))

		--------------test--------------
              precision    recall  f1-score   support

           0       0.73      0.69      0.71     22235
           1       0.71      0.75      0.73     22961

    accuracy                           0.72     45196
   macro avg       0.72      0.72      0.72     45196
weighted avg       0.72      0.72      0.72     45196

		--------------train--------------
              precision    recall  f1-score   support

           0       0.79      0.75      0.77     89214
           1       0.77      0.80      0.79     91568

    accuracy                           0.78    180782
   macro avg       0.78      0.78      0.78    180782
weighted avg       0.78      0.78      0.78    180782



In [92]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=3, n_jobs=-1, random_state=21)
forest.fit(cv_train, y_train)
cv_pred = forest.predict(cv_test)
print('\t\t--------------test--------------')
print(classification_report(y_test, cv_pred))
print('\t\t--------------train--------------')
print(classification_report(y_train, forest.predict(cv_train)))

		--------------test--------------
              precision    recall  f1-score   support

           0       0.64      0.72      0.68     22235
           1       0.69      0.61      0.65     22961

    accuracy                           0.66     45196
   macro avg       0.67      0.67      0.66     45196
weighted avg       0.67      0.66      0.66     45196

		--------------train--------------
              precision    recall  f1-score   support

           0       0.92      0.95      0.93     89214
           1       0.95      0.92      0.93     91568

    accuracy                           0.93    180782
   macro avg       0.93      0.93      0.93    180782
weighted avg       0.93      0.93      0.93    180782



In [93]:
forest.fit(tfidf_train, y_train)
tfidf_pred = forest.predict(tfidf_test)
print('\t\t--------------test--------------')
print(classification_report(y_test, tfidf_pred))
print('\t\t--------------train--------------')
print(classification_report(y_train, forest.predict(tfidf_train)))

		--------------test--------------
              precision    recall  f1-score   support

           0       0.67      0.67      0.67     22235
           1       0.68      0.68      0.68     22961

    accuracy                           0.67     45196
   macro avg       0.67      0.67      0.67     45196
weighted avg       0.67      0.67      0.67     45196

		--------------train--------------
              precision    recall  f1-score   support

           0       0.94      0.93      0.93     89214
           1       0.93      0.94      0.94     91568

    accuracy                           0.93    180782
   macro avg       0.93      0.93      0.93    180782
weighted avg       0.93      0.93      0.93    180782



### Вывод :

две модели лог. регресия и случайный лес переучены не зависемо от метода кодировки
---